In [1]:
import os
# os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd



@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: Path
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    max_features: str
    max_depth: int
    target_column: str

In [3]:
from src.utils.utlis import read_yaml,create_dir
from src.constants import *

ModuleNotFoundError: No module named 'src'

In [14]:
class ConfigManager:
    def __init__(self,
                  config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_dir([self.config.artifacts_root])


    def model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_dir([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators= params.n_estimators,
            min_samples_split= params.min_samples_split,
            min_samples_leaf= params.min_samples_leaf,
            max_features= params.max_features,
            max_depth= params.max_depth,
            target_column= schema.name
            
        )
        

        return model_trainer_config

In [15]:
import pandas as pd
import os
from src.logger.custom_logging import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [16]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        model = RandomForestClassifier(n_estimators= self.config.n_estimators, min_samples_split= self.config.min_samples_split, min_samples_leaf= self.config.min_samples_leaf, max_features=self.config.max_features, max_depth= self.config.max_depth,random_state=42)
        model.fit(train_x, train_y)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [17]:
try:
    config = ConfigManager()
    model_trainer_config = config.model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

/Users/akashmukherjee/Programming/MLOPS/Bappy/Wine quality/my_env/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
